In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  # make plots prettier
import os
import matplotlib.gridspec as gridspec
import glob
from scipy.ndimage import gaussian_filter
import colorcet as cc

def set_plotting_style():
    """
    Formats plotting enviroment to that used in Physical Biology of the Cell,
    2nd edition. To format all plots within a script, simply execute
    `mwc_induction_utils.set_plotting_style() in the preamble.
    """
    rc = {'lines.linewidth': 2,
          'axes.labelsize': 10,
          'axes.titlesize': 9,
          'axes.facecolor': '#E3DCD0',
          'xtick.labelsize': 7,
          'ytick.labelsize': 7,
          # 'font.family': 'Lucida Sans Unicode',
          'grid.linestyle': '-',
          'grid.linewidth': 0.5,
          'grid.color': '#ffffff',
          'legend.fontsize': 12}
    plt.rc('text.latex', preamble=r'\usepackage{sfmath}')
    plt.rc('xtick.major', pad=-1)
    plt.rc('ytick.major', pad=-1)
    plt.rc('mathtext', fontset='stixsans', sf='sansserif')
    plt.rc('figure', figsize=[4.0, 3.0])
    plt.rc('svg', fonttype='none')
    plt.rc('legend', title_fontsize='16', frameon=True, 
           facecolor='#E3DCD0', framealpha=1)
    sns.set_style('darkgrid', rc=rc)
    sns.set_palette("colorblind", color_codes=True)
    sns.set_context('notebook', rc=rc)

set_plotting_style()



def process_piv_data(link, µl, vmax=False, time_int=3, L=100, sigma=5, skip_frames=0, remove_end_frames=0):
    """
    Processes PIV data from a series of text files and calculates various physical quantities.
    
    Parameters:
    link (str): The path to the text files containing the data.
    µl (float): The volume in microliters used in calculations.
    vmax (bool): Whether to use the average of the top 10 velocities or the mean velocity.
    time_int (int): The time interval between measurements, used to calculate time in minutes.
    L (int): The correlation length, used in calculations.
    sigma (float): The standard deviation of the Gaussian kernel, used in the Gaussian filter.
    skip_frames (int): The number of initial frames to skip.
    remove_end_frames (int): The number of final frames to remove.
    
    Returns:
    pd.DataFrame: A DataFrame containing time, velocities, vorticities, divergences, distance, 
                  power, work, drag force, and cumulative drag force.
    """
    
    # Get sorted list of text files, skip the initial frames and remove the end frames as specified
    files = sorted(glob.glob(link))[skip_frames:-remove_end_frames or None]
    
    # Initialize lists to store extracted magnitudes
    velocities, vorticities, divergences = [], [], []
       
    for file in files:
        df = pd.read_csv(file, skiprows=2).fillna(0)
        
        # Determine method to calculate velocity
        if vmax:
            v = df['magnitude [m/s]'].nlargest(50).mean()
        else:
            v = df['magnitude [m/s]'].mean()
        
        velocities.append(v)
        vorticities.append(df["vorticity [1/s]"].mean())
        divergences.append(df["divergence [1/s]"].mean())

    
    # Create a DataFrame with time and processed velocities
    result_df = pd.DataFrame(velocities, columns=["v (m/s)"]).reset_index()
    result_df['time_min'] = result_df['index'] * time_int / 60
    result_df['v (m/s)'] = gaussian_filter(result_df['v (m/s)'] - result_df['v (m/s)'].min(), sigma=sigma)
    
    # Calculate additional physical quantities
    result_df['vorticity (1/s)'], result_df['divergence (1/s)'] = vorticities, divergences
    result_df['distance (m)'] = (result_df['v (m/s)'] * result_df['time_min'].diff()).cumsum()
    
    vol, µ, corr = µl * 1E-9, 1E-3, L * 1E-6  # Convert units for volume, viscosity, and correlation length
    result_df["Power (W)"] = vol * µ * (result_df["v (m/s)"] / corr) ** 2
    result_df['Work (J)'] = (result_df["Power (W)"] * result_df['time_min'].diff()).cumsum()
    
    r = 10e-6  # Radius of the bead in meters
    result_df['F_drag (N)'] = 6 * np.pi * µ * r * result_df['v (m/s)']
    result_df['F'] = (result_df["F_drag (N)"] * result_df['time_min'].diff()).cumsum()
    
    return result_df.iloc[:, 1:]


def plot_velocity(df, label, filename=None, max_frame=None, x_limits=None, y_limits=None):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df = df.iloc[:max_frame,:]
    
    # Use a green color from colorcet
    green_color = cc.glasbey_hv[27]  # Selecting a green color from the glasbey_hv colormap
    
    ax.plot(df['time_min'], df['v (m/s)'] * 1e6, label=label, color=green_color)
    ax.fill_between(df['time_min'], df['v (m/s)'] * 1e6, color=green_color, alpha=0.2)
    
    ax.set_xlabel('time (min)')
    ax.set_ylabel('Velocity (µm/s)')
    ax.set_title('Velocity vs time, K401-Biotin in TXTL')
    ax.legend()
    
    # Set axis limits if provided
    if x_limits:
        ax.set_xlim(x_limits)
    if y_limits:
        ax.set_ylim(y_limits)
    
    if filename:
        plt.savefig(filename, format='jpg', dpi=150)
    else:
        plt.show()
    
    plt.close(fig)




def plot_distance(df, label, filename=None, max_frame=None, x_limits=None, y_limits=None):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df = df.iloc[:max_frame,:]
    green_color = cc.glasbey_hv[27]  # Selecting a green color from the glasbey_hv colormap

    ax.plot(df['time_min'].iloc[-1], df['distance (m)'].iloc[-1] * 1e6, 'o', label=label, markersize=16, color=green_color)
    
    ax.set_xlabel('time (min)')
    ax.set_ylabel('Distance (µm)')
    ax.set_title('Displacement vs time, K401-Biotin in TXTL')

    ax.legend()
    
    if x_limits:
        ax.set_xlim(x_limits)
    if y_limits:
        ax.set_ylim(y_limits)
    
    if filename:
        plt.savefig(filename, format='jpg', dpi=150)
    else:
        plt.show()
    
    plt.close(fig)



def plot_power(df, label, filename=None, max_frame=None, x_limits=None, y_limits=None):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df = df.iloc[:max_frame,:]
    green_color = cc.glasbey_hv[27]  # Selecting a green color from the glasbey_hv colormap

    
    ax.plot(df['time_min'], df['Power (W)'], label=label, color=green_color)
    ax.fill_between(df['time_min'], df['Power (W)'], alpha=0.2, color=green_color)
    
    ax.set_xlabel('time (min)')
    ax.set_ylabel('Power (W)')
    ax.set_title('Power vs time, K401-Biotin in TXTL')

    ax.legend()
    
    if x_limits:
        ax.set_xlim(x_limits)
    if y_limits:
        ax.set_ylim(y_limits)
    
    if filename:
        plt.savefig(filename, format='jpg', dpi=150)
    else:
        plt.show()
    
    plt.close(fig)


def plot_vorticity(df, label, filename=None, max_frame=None, x_limits=None, y_limits=None):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df = df.iloc[:max_frame,:]
    green_color = cc.glasbey_hv[27]  
    
    ax.plot(df['time_min'], df['vorticity (1/s)'], label=label, color=green_color)
    ax.fill_between(df['time_min'], df['vorticity (1/s)'], alpha=0.2, color=green_color)
    
    ax.set_xlabel('time (min)')
    ax.set_ylabel('Vorticity (1/s)')
    ax.set_title('Vorticity vs time, K401-Biotin in TXTL')
    ax.legend()
    
    if x_limits:
        ax.set_xlim(x_limits)
    if y_limits:
        ax.set_ylim(y_limits)
    
    if filename:
        plt.savefig(filename, format='jpg', dpi=150)
    else:
        plt.show()
    
    plt.close(fig)
    

    
def plot_divergence(df, label, filename=None, max_frame=None, x_limits=None, y_limits=None):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df = df.iloc[:max_frame,:]
    green_color = cc.glasbey_hv[27]  
    
    ax.plot(df['time_min'], df['divergence (1/s)'], label=label, color=green_color)
    ax.fill_between(df['time_min'], df['divergence (1/s)'], alpha=0.2, color=green_color)
    
    ax.set_xlabel('time (min)')
    ax.set_ylabel('Divergence (1/s)')
    ax.set_title('Divergence vs time, K401-Biotin in TXTL')
    ax.legend()
    
    if x_limits:
        ax.set_xlim(x_limits)
    if y_limits:
        ax.set_ylim(y_limits)
    
    if filename:
        plt.savefig(filename, format='jpg', dpi=150)
    else:
        plt.show()
    
    plt.close(fig)

In [2]:
link = "../../data/K401bio_protein-PIV_velocity_control/K401bio_protein-PIV_velocity_control.csv"

df = pd.read_csv(link).iloc[:,1:]

FileNotFoundError: [Errno 2] No such file or directory: '../../data/K401bio_protein-PIV_velocity_control/K401bio_protein-PIV_velocity_control.csv'

In [11]:
# Uncomment to generate image sequence of plots per frame
# Define directory name
dir_name = "../../data/K401bio_protein-PIV_velocity_control/plots/plots_velocity"
label = 'K401 0.5µM'

def generate_image_sequence(dir_name, label):
    # Check if directory exists, if not, create it
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    # Loop through each frame of df and save a plot
    for i in range(1, len(df) + 1):
        plot_velocity(
            df.iloc[:i,:], 
            label, 
             x_limits=(-0.5,26), y_limits=(0,0.7),
            filename=f"{dir_name}/velocity_plot_{i}.jpg"
        )
        
generate_image_sequence(dir_name, label)

In [12]:
# Uncomment to generate image sequence of plots per frame

# Define directory name
dir_name = "../../data/K401bio_protein-PIV_velocity_control/plots/plots_distance"

# Check if directory exists, if not, create it
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
    
    
for i in range(1, len(df) + 1):
    plot_distance(
        df.iloc[:i,:], 
        'K401 0.5µM', 
        x_limits=(-0.5,26), y_limits=(-0.025,7),
        filename=f"{dir_name}/distance_plot_{i}.jpg"

    )

In [13]:
# Uncomment to generate image sequence of plots per frame
# Define directory name
dir_name = "../../data/K401bio_protein-PIV_velocity_control/plots/plots_power"

# Check if directory exists, if not, create it
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
    
# Loop through each frame of df and save a plot
for i in range(1, len(df) + 1):
    plot_power(
        df.iloc[:i,:], 
        'K401 0.5µM',  
         x_limits=(-0.5,26), y_limits=(-0.25E-17,1E-16),
        filename=f"{dir_name}/power_plot_{i}.jpg"
    )

In [14]:
## Uncomment to generate image sequence of plots per frame
# Define directory name for vorticity
dir_name_vorticity = "../../data/K401bio_protein-PIV_velocity_control/plots/plots_vorticity"

# Check if directory exists, if not, create it
if not os.path.exists(dir_name_vorticity):
    os.makedirs(dir_name_vorticity)
    
# Loop through each frame of df and save a plot for vorticity
for i in range(1, len(df) + 1):
    plot_vorticity(
        df.iloc[:i,:], 
        'K401 0.5µM', 
        x_limits=(-0.5,26), 
        y_limits=(-0.0007, 0.0007),  # Example y-limits, adjust accordingly
        filename=f"{dir_name_vorticity}/vorticity_plot_{i}.jpg"
    )


In [15]:
## Uncomment to generate image sequence of plots per frame
# Define directory name for divergence
dir_name_divergence = "../../data/K401bio_protein-PIV_velocity_control/plots/plots_divergence"

# Check if directory exists, if not, create it
if not os.path.exists(dir_name_divergence):
    os.makedirs(dir_name_divergence)
    
# Loop through each frame of df and save a plot for divergence
for i in range(1, len(df) + 1):
    plot_divergence(
        df.iloc[:i,:], 
        'K401 0.5µM', 
        x_limits=(-0.5,26), 
        y_limits=(-0.0005, 0.0005),  # Example y-limits, adjust accordingly
        filename=f"{dir_name_divergence}/divergence_plot_{i}.jpg"
    )
